In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [2]:
# Load dataset
df = pd.read_csv(r"dataset\company_01_merge_sales_without_previous.csv")
X = df.drop("sales", axis=1).values
y = df["sales"].values

In [3]:
# Convert '2077-04' like dates to numerical values
# Extract year and month
df['Year'] = df['date'].apply(lambda x: int(x.split('-')[0]))
df['Month'] = df['date'].apply(lambda x: int(x.split('-')[1]))

# Drop original 'Date' column
df.drop('date', axis=1, inplace=True)

# # Normalize remaining numeric data
scaler = StandardScaler()
X = scaler.fit_transform(df[['Year', 'Month']])


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [6]:
class ANNModel(nn.Module):
    def __init__(self, input_dim):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(64, 32)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x

In [7]:
model = ANNModel(input_dim=X.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Training loop
epochs = 1000
best_loss = float("inf")

In [9]:
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    if loss.item() < best_loss:
        best_loss = loss.item()
        torch.save(model.state_dict(), "best_ann_model.pth")

    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [100/1000], Loss: 126980647976697856.0000
Epoch [200/1000], Loss: 126978637932003328.0000
Epoch [300/1000], Loss: 126967883333894144.0000
Epoch [400/1000], Loss: 126935430561005568.0000
Epoch [500/1000], Loss: 126860852748877824.0000
Epoch [600/1000], Loss: 126723061608087552.0000
Epoch [700/1000], Loss: 126476444585951232.0000
Epoch [800/1000], Loss: 126149365646491648.0000
Epoch [900/1000], Loss: 125587532384567296.0000
Epoch [1000/1000], Loss: 124948673179156480.0000


In [10]:
model.load_state_dict(torch.load("best_ann_model.pth"))
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    test_loss = criterion(y_pred, y_test)
    print(f"Test Loss: {test_loss.item():.4f}")

Test Loss: 89112410954137600.0000


C:\Users\binay\AppData\Local\Temp\ipykernel_23276\2434704393.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_ann_model.pth"))


In [13]:
model.load_state_dict(torch.load("best_ann_model.pth"))
model.eval()
with torch.no_grad():
    y_pred=model(X_test)
    test_loss = criterion(y_pred, y_test)
    print(f"Test Loss: {test_loss.item():.4f}")

Test Loss: 89112410954137600.0000


C:\Users\binay\AppData\Local\Temp\ipykernel_23276\1990453997.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_ann_model.pth"))


In [11]:
y_pred

tensor([[4882421.0000],
        [5455045.0000],
        [4029963.0000],
        [3868659.5000],
        [4495774.0000],
        [2343270.0000],
        [2804667.0000],
        [3025029.0000],
        [3101418.5000],
        [3965076.2500],
        [2502437.5000],
        [5116031.0000]])

In [12]:
y_test

tensor([[1.5547e+08],
        [3.7760e+08],
        [4.6597e+08],
        [1.3799e+08],
        [4.6567e+08],
        [3.5807e+08],
        [3.6315e+07],
        [3.2236e+08],
        [2.2944e+08],
        [5.3520e+07],
        [3.8274e+08],
        [1.9543e+08]])